## Load SII Data

In [1]:
import numpy as np
import csv
import os
import pickle

# Generate data in different types
data_training_dir = "dfalc_data/training/"
data_testing_dir = "dfalc_data/testing/"
zero_distance_threshold = 6
number_of_features = 65
name = "animal"
types = {}
add_noise_flag = True
mask_rate =0.1
with open("classes.csv", "r") as f:
    cnt = 0
    for line in f.readlines():
        types[line.strip()] = cnt
        cnt += 1
        
        
if name == "vehicle":
    # uncomment this line for training the vehicle object types
    selected_types_name = np.array(['aeroplane','artifact_wing','body','engine','stern','wheel','bicycle','chain_wheel','handlebar','headlight','saddle','bus','bodywork','door','license_plate','mirror','window','car','motorbike','train','coach','locomotive','boat'])

if name == "indoor":
    # uncomment this line for training the indoor object types
    selected_types_name = np.array(['bottle','body','cap','pottedplant','plant','pot','tvmonitor','screen']) #'chair','sofa','diningtable'

if name == "animal":
    # uncomment this line for training the animal object types
    selected_types_name = np.array(['person','arm','ear','ebrow','foot','hair','hand','mouth','nose','eye','head','leg','neck','torso','cat','tail','bird','animal_wing','beak','sheep','horn','muzzle','cow','dog','horse','hoof'])



selected_types = [types[n] for n in selected_types_name]

# uncomment this line for training all the object types
# selected_types = list(range(len(types)))

def add_noise(data, mask_rate = 0.2):
    nosied_data = np.array(data, copy=True)
    coords = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            coords.append([i,j])

    size = len(coords)
    print("cEmb masked size: ", int(mask_rate*size))
    for i in np.random.choice(size, int(mask_rate*size), replace=False):
        x,y = coords[i]
        nosied_data[x,y] = np.random.uniform(0,1)
    return nosied_data

def containment_ratios_between_two_bbxes(bb1, bb2):
    bb1_area = (bb1[-2] - bb1[-4]) * (bb1[-1] - bb1[-3])
    bb2_area = (bb2[-2] - bb2[-4]) * (bb2[-1] - bb2[-3])
    w_intersec = max(0,min([bb1[-2], bb2[-2]]) - max([bb1[-4], bb2[-4]]))
    h_intersec = max(0,min([bb1[-1], bb2[-1]]) - max([bb1[-3], bb2[-3]]))
    bb_area_intersection = w_intersec * h_intersec
    return [float(bb_area_intersection)/bb1_area, float(bb_area_intersection)/bb2_area]

def get_data(train_or_test_swritch,max_rows=10000000):
    assert train_or_test_swritch == "train" or train_or_test_swritch == "test"

    # Fetching the data from the file system

    if train_or_test_swritch == "train":
        data_dir = data_training_dir
    if train_or_test_swritch == "test":
        data_dir = data_testing_dir
    data = np.genfromtxt(data_dir+"features.csv",delimiter=",",max_rows=max_rows)
    if add_noise_flag:
        if os.path.exists("dfalc_data/training/masked"+str(mask_rate)+"_type.pkl"):
            data = pickle.load(open("dfalc_data/training/masked"+str(mask_rate)+"_type.pkl","rb"))
        else:
            data = add_noise(data,mask_rate)
            pickle.dump(data,open("dfalc_data/training/masked"+str(mask_rate)+"_type.pkl","wb"),2)
    types_of_data = np.genfromtxt(data_dir + "types.csv", dtype="i", max_rows=max_rows)
    idx_whole_for_data = np.genfromtxt(data_dir+ "partOf.csv",dtype="i",max_rows=max_rows)
    idx_of_cleaned_data = np.where(np.logical_and(
        np.all(data[:, -2:] - data[:, -4:-2] >= zero_distance_threshold, axis=1),
        np.in1d(types_of_data,selected_types)))[0]
    print("deleting", len(data) - len(idx_of_cleaned_data), "small bb out of", data.shape[0], "bb")
    data = data[idx_of_cleaned_data]
    data[:, -4:] /= 500

    # Cleaning data by removing small bounding boxes and recomputing indexes of partof data

    types_of_data = types_of_data[idx_of_cleaned_data]
    idx_whole_for_data = idx_whole_for_data[idx_of_cleaned_data]
    for i in range(len(idx_whole_for_data)):
        if idx_whole_for_data[i] != -1 and idx_whole_for_data[i] in idx_of_cleaned_data:
            idx_whole_for_data[i] = np.where(idx_whole_for_data[i] == idx_of_cleaned_data)[0]
        else:
            idx_whole_for_data[i] = -1

    # Grouping bbs that belong to the same picture

    pics = {} #记录了每张图片对应的bbox，即data的id
    for i in range(len(data)):
        if data[i][0] in pics:
            pics[data[i][0]].append(i)
        else:
            pics[data[i][0]] = [i]

    pairs_of_data = np.array(
        [np.concatenate((data[i][1:], data[j][1:], containment_ratios_between_two_bbxes(data[i], data[j]))) for p in
         pics for i in pics[p] for j in pics[p]])

    pairs_of_bb_idxs = np.array([(i,j) for p in pics for i in pics[p] for j in pics[p]]) #枚举同一张图片里不同objects(bbox) pair

    partOf_of_pair_of_data = np.array([idx_whole_for_data[i] == j for p in pics for i in pics[p] for j in pics[p]])

    return data, pairs_of_data, types_of_data, partOf_of_pair_of_data, pairs_of_bb_idxs, pics


data, pairs_of_data, types_of_data, partOf_of_pairs_of_data, pairs_of_bb_idxs, pics = get_data("test",max_rows=1000000000)
print("individual size: ", data.shape[0], "concept size: ", data.shape[1]-4)



deleting 16719 small bb out of 36218 bb
individual size:  19499 concept size:  61


## Generate initialization

In [2]:
from numpy import require
from Dataset import OntologyDataset
from model import DFALC, DFALC2
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from torch.utils.data.sampler import RandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

conceptSize = len(selected_types)
roleSize = 1
individualSize = data.shape[0]
cEmb_candid = torch.Tensor(data[:,1:-4])
rEmb_candid = torch.zeros(1, individualSize, individualSize)
partOf_of_pairs_idx = np.where(partOf_of_pairs_of_data)[0]
for idx,(i,j) in enumerate(pairs_of_bb_idxs):
    i_partof_j_p, j_partof_i_p = pairs_of_data[idx][-2], pairs_of_data[idx][-1]
    if i_partof_j_p == 1: j_partof_i_p = 0
    if j_partof_i_p == 1: i_partof_j_p = 0
    rEmb_candid[0,i,j] = i_partof_j_p
    rEmb_candid[0,j,i] = j_partof_i_p

info_path = "dfalc_data"
file_name = "PascalPartOntology_"+name+".owl"
with open(os.path.join(info_path,file_name+"_roles.txt"),"w") as f:
    f.write("http://www.w3.org/2002/07/partOf")
with open(os.path.join(info_path,file_name+"_individuals.txt"),"w") as f:
    individuals = []
    for p in pics:
        for i in pics[p]:
            f.write(str(i) + "\n")

params = {
        "conceptPath": os.path.join(info_path,file_name+"_concepts.txt"),
        "rolePath": os.path.join(info_path,file_name+"_roles.txt"),
        "individualPath": os.path.join(info_path,file_name+"_individuals.txt"),
        "normalizationPath": os.path.join(info_path,file_name+"_normalization.txt"),
        "batchSize": 3,
        "epochSize":10,
        "earlystopping":10,
        "dist": "minkowski",
        "norm":1,
        "norm_rate":0.5,
        "norm_rate2":0
    }
to_train = False

save_path = "dfalc_data"
if to_train: save_path = os.path.join(save_path,"training")
else: save_path = os.path.join(save_path,"testing")
save_path += "/PascalPartOntology_"
dataset = OntologyDataset(params,save_path)

cEmb_init = torch.zeros(dataset.conceptSize-2, individualSize)
rEmb_init = torch.zeros(1, individualSize, individualSize)
# cEmb_init.fill_(0.5)
# rEmb_init.fill_(0.5)

true_rEmb = torch.zeros(1, individualSize, individualSize)
for idx, (i,j) in enumerate(pairs_of_bb_idxs[partOf_of_pairs_of_data]):
#     ci = dataset.concept2id['http://www.w3.org/2002/07/'+id2types[types_of_data[i]]]
#     cj = dataset.concept2id['http://www.w3.org/2002/07/'+id2types[types_of_data[j]]]
    true_rEmb[0,i,j] = 1 #
    rEmb_init[0,i,j] = rEmb_candid[0,i,j]
# rEmb_init = rEmb_candid.clone()



def get_definers_initial_semantics(cid, use_partOf=False):
    print(cid)
    if use_partOf:
        emb = torch.zeros(1,individualSize)
        print("mode: 5")
        dataset.mode = 5
        for left,right,neg in dataset:
            print(left, right,neg)
            if right.item() == cid:
                emb = torch.max(torch.minimum(true_rEmb[0],cEmb_init[left[1]].unsqueeze(1).expand(true_rEmb[0].shape)),1).values + 0.1
                break
        dataset.mode = 0
        print("mode: 0")
        
        for left,right,neg in dataset:
            print(left, right,neg)
            if right.item() == cid:
                emb = torch.maximum(cEmb_init[left.item()], emb) + 0.1
       
        return emb
    else:
        dataset.mode = 2
        for left,right,neg in dataset:
            if left == cid:
                if id2concept[right[0].item()][:7] == "definer":
                    return torch.maximum(get_definers_initial_semantics(right[0],use_partOf=False),cEmb_init[right[1]]) - 0.1
                return torch.maximum(cEmb_init[right[0]],cEmb_init[right[1]]) - 0.1
                
    
    return torch.zeros(1,individualSize)

cid2typeid = {} 
interest_cids = []
cnt = 0
id2concept = {c:idx for idx, c in dataset.concept2id.items()}

                
for c,idx in dataset.concept2id.items():
    cname = c.replace("http://www.w3.org/2002/07/","")
    if cname in selected_types_name:
        cEmb_init[idx] = cEmb_candid[:,types[cname]]
        cid2typeid[cnt] = types[cname]
        interest_cids.append(True)
        cnt += 1
    else:
        interest_cids.append(False)
interest_cids = interest_cids[:-2]
for c,idx in dataset.concept2id.items():
    cname = c.replace("http://www.w3.org/2002/07/","")
    if cname in selected_types_name: continue
    if (idx == dataset.conceptSize -3) or (idx == dataset.conceptSize-4):continue
    cEmb_init[idx] = get_definers_initial_semantics(idx,use_partOf=True)
    print(torch.max(cEmb_init[idx]))
        
cEmb_init[-1] = torch.ones(1,individualSize)
cEmb_init[-2] = torch.zeros(1,individualSize)

id2types = {idx:name for name,idx in types.items()}






{'http://www.w3.org/2002/07/ebrow': 0, 'http://www.w3.org/2002/07/horn': 1, 'http://www.w3.org/2002/07/tail': 2, 'http://www.w3.org/2002/07/torso': 3, 'http://www.w3.org/2002/07/sheep': 4, 'http://www.w3.org/2002/07/leg': 5, 'http://www.w3.org/2002/07/bird': 6, 'http://www.w3.org/2002/07/horse': 7, 'http://www.w3.org/2002/07/cat': 8, 'http://www.w3.org/2002/07/ear': 9, 'http://www.w3.org/2002/07/arm': 10, 'http://www.w3.org/2002/07/animal_wing': 11, 'http://www.w3.org/2002/07/beak': 12, 'http://www.w3.org/2002/07/hand': 13, 'http://www.w3.org/2002/07/cow': 14, 'http://www.w3.org/2002/07/foot': 15, 'http://www.w3.org/2002/07/muzzle': 16, 'http://www.w3.org/2002/07/mouth': 17, 'http://www.w3.org/2002/07/neck': 18, 'http://www.w3.org/2002/07/person': 19, 'http://www.w3.org/2002/07/eye': 20, 'http://www.w3.org/2002/07/head': 21, 'http://www.w3.org/2002/07/nose': 22, 'http://www.w3.org/2002/07/hair': 23, 'http://www.w3.org/2002/07/dog': 24, 'http://www.w3.org/2002/07/hoof': 25, 'owl:Nothing

In [3]:
id2concept

{0: 'http://www.w3.org/2002/07/ebrow',
 1: 'http://www.w3.org/2002/07/horn',
 2: 'http://www.w3.org/2002/07/tail',
 3: 'http://www.w3.org/2002/07/torso',
 4: 'http://www.w3.org/2002/07/sheep',
 5: 'http://www.w3.org/2002/07/leg',
 6: 'http://www.w3.org/2002/07/bird',
 7: 'http://www.w3.org/2002/07/horse',
 8: 'http://www.w3.org/2002/07/cat',
 9: 'http://www.w3.org/2002/07/ear',
 10: 'http://www.w3.org/2002/07/arm',
 11: 'http://www.w3.org/2002/07/animal_wing',
 12: 'http://www.w3.org/2002/07/beak',
 13: 'http://www.w3.org/2002/07/hand',
 14: 'http://www.w3.org/2002/07/cow',
 15: 'http://www.w3.org/2002/07/foot',
 16: 'http://www.w3.org/2002/07/muzzle',
 17: 'http://www.w3.org/2002/07/mouth',
 18: 'http://www.w3.org/2002/07/neck',
 19: 'http://www.w3.org/2002/07/person',
 20: 'http://www.w3.org/2002/07/eye',
 21: 'http://www.w3.org/2002/07/head',
 22: 'http://www.w3.org/2002/07/nose',
 23: 'http://www.w3.org/2002/07/hair',
 24: 'http://www.w3.org/2002/07/dog',
 25: 'http://www.w3.org/20

## Baseline Performance

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
p_baseline = torch.argmax(cEmb_init[interest_cids],0).detach().numpy()
h_baseline = np.array([cid2typeid[i] for i in p_baseline])

y_true = []
y_baseline = []
# partof_true = np.ravel(true_rEmb.numpy()).T
# partof_baseline = (np.ravel(rEmb_init.numpy()).T > .5).astype(int)
for idx,t in enumerate(types_of_data):
    if t in selected_types:
        y_baseline.append(selected_types_name[np.where(selected_types==h_baseline[idx])[0][0]])
        y_true.append(selected_types_name[np.where(selected_types==t)[0][0]])



# confusion_matrix(y_true, y_pred, labels=selected_types_name)
p,r,f,s = precision_recall_fscore_support(y_true, y_baseline)
print(precision_recall_fscore_support(y_true, y_baseline,average='macro'))
print({selected_types_name[i]:s[i] for i in range(len(selected_types))})
# print("PartOf: ", precision_recall_fscore_support(partof_true, partof_baseline))

(0.329359216146057, 0.40252377249670135, 0.32682017980119, None)
{'person': 77, 'arm': 1705, 'ear': 124, 'ebrow': 154, 'foot': 194, 'hair': 89, 'hand': 236, 'mouth': 1500, 'nose': 379, 'eye': 1043, 'head': 670, 'leg': 862, 'neck': 1160, 'torso': 2014, 'cat': 113, 'tail': 44, 'bird': 105, 'animal_wing': 2680, 'beak': 455, 'sheep': 404, 'horn': 996, 'muzzle': 806, 'cow': 1233, 'dog': 106, 'horse': 326, 'hoof': 2024}


## Type Classification - Revising based on DF-ALC with rule-based loss

In [5]:
from torch.optim.lr_scheduler import LambdaLR
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
device = torch.device("cuda:0")

import pickle
params = {
        "conceptPath": os.path.join(info_path,file_name+"_concepts.txt"),
        "rolePath": os.path.join(info_path,file_name+"_roles.txt"),
        "individualPath": os.path.join(info_path,file_name+"_individuals.txt"),
        "normalizationPath": os.path.join(info_path,file_name+"_normalization.txt"),
        "batchSize": 5,
        "epochSize":10,
        "earlystopping":10,
        "dist": "minkowski",
        "norm":1,
        "norm_rate":0.3,
        "norm_rate2":0.3,
        "alpha": 0.8
    }



nEpoch = 15000
batchSz = 4
best_loss = 100
last_best_epoch = 0
patience = 1
best_f1 = 0
best_cEmb = None

model = DFALC(params, conceptSize, roleSize, cEmb_init, rEmb_init, device).to(device)
model = nn.DataParallel(model, device_ids=[0,1])
# model= nn.DataParallel(model,device_ids=[0, 1, 2,3])
optimizer = optim.Adam(model.parameters(), 2e-4)
# scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.65 ** epoch)

for epoch in range(1, nEpoch+1):
    loss_final, data_cnt = 0, 0
#     losses = []
    for mode in range(7):
        dataset.mode = mode
        if len(dataset) == 0: continue
#         print(mode, len(dataset))
#         if mode ==3 or mode == 4: continue
        loader = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size=batchSz)
        data_cnt += len(dataset)
        for i, batch in enumerate(loader):
            ptriplets = [b.to(device) for b in batch]
            loss = model(ptriplets, mode, device)#, is_input.contiguous()
#             losses.append(loss)
#             if loss.item()<1e-6: break
            optimizer.zero_grad()
            torch.mean(loss).backward(retain_graph=True)
            optimizer.step()
#         if mode %2 == 0:
#             loss = losses[0]
# #             print(losses)
#             optimizer.zero_grad()
#             if mode > 1:
#                 loss = torch.mean(torch.sum(torch.concat(losses,0).to(device),1))
#             else:
#                 for i in range(1,len(losses)):
#                     loss += losses[i]
#             loss.backward(retain_graph=True)
#             optimizer.step()
#             losses = []
#             data_cnt = 0
#     scheduler.step(losses)
#             losses.append(loss)
    

        # err = computeErr(preds_selected.data.detach(), boardSz, unperm)/batchSz
#     loss = losses[0]
#     for i in range(1,len(losses)):
#         loss += losses[i]
    
    
    
    p = torch.argmax(model.module.cEmb[torch.BoolTensor(interest_cids)],0).detach().cpu().numpy()
    h = np.array([cid2typeid[i] for i in p])
    y_pred = []
    for idx,t in enumerate(types_of_data):
        if t in selected_types:
            y_pred.append(selected_types_name[np.where(selected_types==h[idx])[0][0]])
    
    precision, recall, fbeta_score, support = precision_recall_fscore_support(y_true, y_pred,average='macro')
    print("Epoch {} loss {:6f} precision {} recall {} f1 {}".format(epoch,torch.mean(loss).item(),precision, recall, fbeta_score))
    if fbeta_score > best_f1:
        best_f1 = fbeta_score
        best_cEmb = model.module.cEmb.detach().cpu()
        # err_final += err
#     if loss_final < best_loss:
#         best_loss = loss_final
#         last_best_epoch = epoch
#     else:
#         patience -= 1
#     if patience == 0:
#         break
    
    # loss_final, err_final = loss_final/len(loader), err_final/len(loader)
    # scheduler.step(err_final/len(loader))
#     if not to_train:
#         print('TESTING SET RESULTS: Average loss: {:.4f}'.format(loss_final))
#     if loss_final < 0.05: break

#print('memory: {:.2f} MB, cached: {:.2f} MB'.format(torch.cuda.memory_allocated()/2.**20, torch.cuda.memory_cached()/2.**20))

# pickle.dump(best_cEmb,open("/data/data_wuxuan/SII/dfalc_data/testing/cEmb_"+name+".pkl","wb"))
# pickle.dump(model.rEmb,open("/data/data_wuxuan/SII/dfalc_data/testing/rEmb_"+name+".pkl","wb"))
# pickle.dump(cEmb_candid,open("/data/data_wuxuan/SII/dfalc_data/testing/true_cEmb_"+name+".pkl","wb"))

/home/wuxuan/SII/model.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.cEmb = nn.Parameter(torch.tensor(cEmb_init))
/home/wuxuan/SII/model.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.rEmb = nn.Parameter(torch.tensor(rEmb_init))
/home/wuxuan/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1 loss -2.849978 precision 0.32946526812549426 recall 0.4029559246142467 f1 0.3269560985349171


/home/wuxuan/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 2 loss -3.656815 precision 0.3294856998810112 recall 0.40298711807609716 f1 0.32698839449726425


/home/wuxuan/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

Epoch 625 loss -0.034979 precision 0.3305844418840843 recall 0.40301663317097947 f1 0.325903389177674


/home/wuxuan/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 626 loss -0.054253 precision 0.3305885832588522 recall 0.40298543970912903 f1 0.3258693766859909


/home/wuxuan/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

## Add Noise to PartOf Relation

In [10]:
from numpy import require
from Dataset import OntologyDataset
from model import DFALC, DFALC2
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from torch.utils.data.sampler import RandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

conceptSize = len(selected_types)
roleSize = 1
individualSize = data.shape[0]
cEmb_candid = torch.Tensor(data[:,1:-4])
rEmb_candid = torch.zeros(1, individualSize, individualSize)
partOf_of_pairs_idx = np.where(partOf_of_pairs_of_data)[0]
for idx,(i,j) in enumerate(pairs_of_bb_idxs):
    i_partof_j_p, j_partof_i_p = pairs_of_data[idx][-2], pairs_of_data[idx][-1]
    if i_partof_j_p == 1: j_partof_i_p = 0
    if j_partof_i_p == 1: i_partof_j_p = 0
    rEmb_candid[0,i,j] = i_partof_j_p
    rEmb_candid[0,j,i] = j_partof_i_p

info_path = "dfalc_data"
file_name = "PascalPartOntology_"+name+"_e.owl"
with open(os.path.join(info_path,file_name+"_roles.txt"),"w") as f:
    f.write("http://www.w3.org/2002/07/partOf")
with open(os.path.join(info_path,file_name+"_individuals.txt"),"w") as f:
    individuals = []
    for p in pics:
        for i in pics[p]:
            f.write(str(i) + "\n")

params = {
        "conceptPath": os.path.join(info_path,file_name+"_concepts.txt"),
        "rolePath": os.path.join(info_path,file_name+"_roles.txt"),
        "individualPath": os.path.join(info_path,file_name+"_individuals.txt"),
        "normalizationPath": os.path.join(info_path,file_name+"_normalization.txt"),
        "batchSize": 3,
        "epochSize":10,
        "earlystopping":10,
        "dist": "minkowski",
        "norm":1,
        "norm_rate":0.5,
        "norm_rate2":0
    }
to_train = False

save_path = "dfalc_data"
if to_train: save_path = os.path.join(save_path,"training")
else: save_path = os.path.join(save_path,"testing")
save_path += "/PascalPartOntology_"
dataset = OntologyDataset(params,save_path)

cEmb_init = torch.zeros(dataset.conceptSize-2, individualSize)
rEmb_init = torch.zeros(1, individualSize, individualSize)
# cEmb_init.fill_(0.5)
# rEmb_init.fill_(0.5)

true_rEmb = torch.zeros(1, individualSize, individualSize)
for idx, (i,j) in enumerate(pairs_of_bb_idxs[partOf_of_pairs_of_data]):
#     ci = dataset.concept2id['http://www.w3.org/2002/07/'+id2types[types_of_data[i]]]
#     cj = dataset.concept2id['http://www.w3.org/2002/07/'+id2types[types_of_data[j]]]
    true_rEmb[0,i,j] = 1 #
    rEmb_init[0,i,j] = rEmb_candid[0,i,j]
# rEmb_init = rEmb_candid.clone()



def get_definers_initial_semantics(cid, use_partOf=False):
    print(cid)
    if use_partOf:
        emb = torch.zeros(1,individualSize)
        print("mode: 5")
        dataset.mode = 5
        for left,right,neg in dataset:
            print(left, right,neg)
            if right.item() == cid:
                emb = torch.max(torch.minimum(true_rEmb[0],cEmb_init[left[1]].unsqueeze(1).expand(true_rEmb[0].shape)),1).values + 0.1
                break
        dataset.mode = 0
        print("mode: 0")
        
        for left,right,neg in dataset:
            print(left, right,neg)
            if right.item() == cid:
                emb = torch.maximum(cEmb_init[left.item()], emb) + 0.1
       
        return emb
    else:
        dataset.mode = 2
        for left,right,neg in dataset:
            if left == cid:
                if id2concept[right[0].item()][:7] == "definer":
                    return torch.maximum(get_definers_initial_semantics(right[0],use_partOf=False),cEmb_init[right[1]]) - 0.1
                return torch.maximum(cEmb_init[right[0]],cEmb_init[right[1]]) - 0.1
                
    
    return torch.zeros(1,individualSize)

cid2typeid = {} 
interest_cids = []
cnt = 0
id2concept = {c:idx for idx, c in dataset.concept2id.items()}

                
for c,idx in dataset.concept2id.items():
    cname = c.replace("http://www.w3.org/2002/07/","")
    if cname in selected_types_name:
        cEmb_init[idx] = cEmb_candid[:,types[cname]]
        cid2typeid[cnt] = types[cname]
        interest_cids.append(True)
        cnt += 1
    else:
        interest_cids.append(False)
interest_cids = interest_cids[:-2]
for c,idx in dataset.concept2id.items():
    cname = c.replace("http://www.w3.org/2002/07/","")
    if cname in selected_types_name: continue
    if (idx == dataset.conceptSize -3) or (idx == dataset.conceptSize-4):continue
    cEmb_init[idx] = get_definers_initial_semantics(idx,use_partOf=True)
    print(torch.max(cEmb_init[idx]))
        
cEmb_init[-1] = torch.ones(1,individualSize)
cEmb_init[-2] = torch.zeros(1,individualSize)

id2types = {idx:name for name,idx in types.items()}






{'http://www.w3.org/2002/07/screen': 0, 'http://www.w3.org/2002/07/tvmonitor': 1, 'http://www.w3.org/2002/07/plant': 2, 'http://www.w3.org/2002/07/pottedplant': 3, 'definer_1': 4, 'definer_2': 5, 'http://www.w3.org/2002/07/bottle': 6, 'definer_3': 7, 'definer_4': 8, 'http://www.w3.org/2002/07/cap': 9, 'http://www.w3.org/2002/07/pot': 10, 'http://www.w3.org/2002/07/body': 11, 'owl:Nothing': 12, 'http://www.w3.org/2002/07/owl#Nothing': 12, 'owl:Thing': 13, 'http://www.w3.org/2002/07/owl#Thing': 13}
4
mode: 5
tensor([0, 1]) tensor(0) tensor([0, 0])
tensor([0, 3]) tensor(4) tensor([0, 0])
mode: 0
tensor(3) tensor(13) tensor([0, 0])
tensor(9) tensor(13) tensor([0, 0])
tensor(11) tensor(13) tensor([0, 0])
tensor(1) tensor(13) tensor([0, 0])
tensor(6) tensor(13) tensor([0, 0])
tensor(2) tensor(13) tensor([0, 0])
tensor(10) tensor(13) tensor([0, 0])
tensor(0) tensor(13) tensor([0, 0])
tensor(2) tensor(4) tensor([0, 0])
tensor(10) tensor(4) tensor([0, 0])
tensor(2) tensor(5) tensor([0, 0])
tens

## Add Noise

In [13]:
import sklearn
def add_noise(cEmb, rEmb, mask_rate = 0.2, mask_rEmb = False, mask_cEmb = False):
    masked_cEmb, masked_rEmb = np.array(cEmb, copy=True), np.array(rEmb, copy=True)
    if mask_cEmb:
        coords = []
        for i in range(conceptSize):
            for j in range(individualSize):
                coords.append([i,j])

        size = len(coords)
        print("cEmb masked size: ", int(mask_rate*size))
        for i in np.random.choice(size, int(mask_rate*size), replace=False):
            x,y = coords[i]
            masked_cEmb[x,y] = np.random.uniform(0,1)

    if mask_rEmb:
        coords = set()

        size = individualSize*individualSize
        print("rEmb masked size: ", int(mask_rate*size))
        for i in np.random.choice(individualSize*individualSize, int(mask_rate*size), replace=False):
            x,y,z = np.random.randint(0,high=roleSize), np.random.randint(0,high=individualSize), np.random.randint(0,high=individualSize)
            while (x,y,z) in coords:
                x,y,z = np.random.randint(0,high=roleSize), np.random.randint(0,high=individualSize), np.random.randint(0,high=individualSize)
            coords.add((x,y,z))
    #         if masked_rEmb[x,y,z] == 0:
            v = np.random.uniform(0,1)

    #         while (v == -1) or (v == 1):
    #             v = np.random.uniform(1-self.alpha,self.alpha)
            masked_rEmb[x,y,z] = v
    return torch.Tensor(masked_cEmb), torch.Tensor(masked_rEmb)
noised_cEmb,noised_rEmb = add_noise(cEmb_init.numpy(),rEmb_init.numpy(),0.1,False,True)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
p_baseline = torch.argmax(noised_cEmb[interest_cids],0).detach().numpy()
h_baseline = np.array([cid2typeid[i] for i in p_baseline])

y_true = []
y_baseline = []
partof_true = np.ravel(true_rEmb.numpy()).T
partof_baseline = (np.ravel(noised_rEmb.numpy()).T > .5).astype(int)
for idx,t in enumerate(types_of_data):
    if t in selected_types:
        y_baseline.append(selected_types_name[np.where(selected_types==h_baseline[idx])[0][0]])
        y_true.append(selected_types_name[np.where(selected_types==t)[0][0]])



# confusion_matrix(y_true, y_pred, labels=selected_types_name)
p,r,f,s = precision_recall_fscore_support(y_true, y_baseline)
print(precision_recall_fscore_support(y_true, y_baseline,average='macro'))
print({selected_types_name[i]:s[i] for i in range(len(selected_types))})
print("PartOf: ", precision_recall_fscore_support(partof_true, partof_baseline),sklearn.metrics.accuracy_score(partof_true,partof_baseline))

cEmb masked size:  1144
(0.642654470681203, 0.6468939642703688, 0.6348787640144741, None)
{'bottle': 348, 'body': 202, 'cap': 143, 'pottedplant': 159, 'plant': 154, 'pot': 163, 'tvmonitor': 127, 'screen': 135}
PartOf:  (array([0.99998681, 1.        ]), array([1.        , 0.96556122]), array([0.9999934 , 0.98247891]), array([2046977,     784])) 0.9999868148675554


## Part-of Relation Classification - Revising based on DF-ALC with rule-based loss